In [392]:
import os
import pandas as pd
import numpy as np
import json

# Data Processing

In [393]:
DATA_ROOT = "../data"
PRODUCT_PATH = os.path.join(DATA_ROOT, 'product_detail/json')
BS_PATH = os.path.join(DATA_ROOT, 'bestseller/bsData')

print(f'Product Detail Dir: {os.listdir(PRODUCT_PATH)[0:5]}')
print(f'Best Seller Dir: {os.listdir(BS_PATH)[0:5]}')

Product Detail Dir: ['B071FXZBMV.json', 'B096WLN8XJ.json', 'B09B2QR6LR.json', 'B09T3P7MJ9.json', 'B003I7HS5M.json']
Best Seller Dir: ['industrial_2.json', 'magazines_2.json', 'musical_2.json', 'sports_2.json', 'electronics_1.json']


## Product Detail Data

In [394]:
product_rows = []
product_overview = {}
for filename in os.listdir(PRODUCT_PATH):
    with open(f'{PRODUCT_PATH}/{filename}', 'r') as f:
        product = json.load(f)
        id_, overview = product['id'], product['overview']
        product_overview[id_] = overview
        # remove overview from the product dict
        product.pop('overview')
        # title is the same as id
        product.pop('title')
        # store product in a pandas dataframe
        product_rows.append(product)        


In [395]:
df_prod = pd.DataFrame(product_rows)
print('column data type:\n', df_prod.dtypes)

print('number of rows: ', len(df_prod))
print('number of unique id: ', len(df_prod['id'].unique()))
# Primary Key
assert len(df_prod) == len(df_prod['id'].unique())

df_prod.head()

column data type:
 id               object
price           float64
discount        float64
country          object
quantity          int64
missOverview       bool
missDiscount       bool
missCountry        bool
missQuantity       bool
missPrice          bool
dtype: object
number of rows:  786
number of unique id:  786


,id,price,discount,country,quantity,missOverview,missDiscount,missCountry,missQuantity,missPrice
0,B071FXZBMV,8.99,0.0,US,30,False,True,False,False,False
1,B096WLN8XJ,44.00,0.0,US,2,True,True,False,False,False
2,B09B2QR6LR,17.90,0.0,US,7,True,True,False,False,False
3,B09T3P7MJ9,16.25,0.0,US,13,False,True,False,False,False
4,B003I7HS5M,0.00,0.0,US,0,True,True,False,False,True


In [396]:
product_overview['B071FXZBMV']

{'_Color_': 'Black',
 '_Brand_': 'OHill',
 '_Item_Weight_': '0.15 Pounds',
 '_Binding_': 'Unknown Binding'}

## Best Seller Data

In [397]:
bs_rows = []
bs_overview = {}
for filename in os.listdir(BS_PATH):
    with open(f'{BS_PATH}/{filename}', 'r') as f:
        bs = json.load(f)
        # remove child_categories, next and url from the list
        bs.pop('child_categories')
        bs.pop('next')
        bs.pop('url')
        # now only category and product_listings left
        _category, _product_listings = bs['category'], bs['product_listings']
        for product_listing in _product_listings:
            _id, _rank, _name, _ratings_count, _rating = product_listing['asin'], product_listing['rank'], product_listing['name'], product_listing[
                'ratings_count'], product_listing['rating']
            if _id == None:
                continue
            temp = [_id, _rank, _name, _ratings_count, _rating, _category]
            bs_rows.append(temp)


In [398]:
df_bs = pd.DataFrame(bs_rows)
df_bs.columns = ['id', 'rank', 'name', 'ratings_count', 'rating', 'category']

# convert data type
df_bs['rating'] = df_bs['rating'].astype(float)
df_bs['ratings_count'] = df_bs['ratings_count'].astype(float)
print('column data type:\n', df_bs.dtypes)

# sort data 
df_bs = df_bs.sort_values(by=['category', 'rank'])
df_bs.head()

column data type:
 id                object
rank               int64
name              object
ratings_count    float64
rating           float64
category          object
dtype: object


,id,rank,name,ratings_count,rating,category
397,B079V67BFW,1,"Pampers Choose Your Count, Sensitive Water Bas...",42199.0,4.8,Baby
398,B07SVHLGV8,2,"Diapers Newborn/Size 0 (< 10 lb), 140 Count - ...",54499.0,4.8,Baby
399,B07MB5RY9N,3,"Baby Wipes, Huggies Natural Care Sensitive Bab...",70559.0,4.8,Baby
400,B0779Z53SD,4,"WaterWipes Original Baby Wipes, 99.9% Water, U...",1408.0,4.8,Baby
401,B08QRKY3NJ,5,"Baby Wipes, Unscented, Huggies Simply Clean Fr...",18881.0,4.8,Baby


In [399]:
print('number of rows: ', len(df_bs))
print('number of unique id: ', len(df_bs['id'].unique()))
# print duplicate data
mask = df_bs['id'].duplicated(keep=False)
print('Duplicated Rows:')
df_bs[mask].sort_values('id')
# Drop duplicate
df_bs = df_bs.drop_duplicates(subset=['id'], keep='first')
print('number of rows: ', len(df_bs))
print('number of unique id: ', len(df_bs['id'].unique()))
assert len(df_bs['id']) == len(df_bs['id'].unique())

number of rows:  994
number of unique id:  986
Duplicated Rows:
number of rows:  986
number of unique id:  986


In [400]:
# give each category an index


## Join two tables & Define top seller

In [401]:
# join product and bestseller table
df = pd.merge(df_prod, df_bs, on="id")
df.head()

,id,price,discount,country,quantity,missOverview,missDiscount,missCountry,missQuantity,missPrice,rank,name,ratings_count,rating,category
0,B071FXZBMV,8.99,0.0,US,30,False,True,False,False,False,31,"OHill Cable Clips,16 Pack Black Adhesive Cord ...",45032.0,4.5,Electronics
1,B096WLN8XJ,44.00,0.0,US,2,True,True,False,False,False,38,2021 Topps Series 2 Baseball Blaster Box,203.0,4.7,Sports Collectibles
2,B09B2QR6LR,17.90,0.0,US,7,True,True,False,False,False,81,2021 Panini Prizm Draft Picks Football CELLO p...,21.0,4.1,Sports Collectibles
3,B09T3P7MJ9,16.25,0.0,US,13,False,True,False,False,False,7,"Kitchens Slow Cooker Liners, Regular (Fits 3-8...",NaN,NaN,Kitchen & Dining
4,B003I7HS5M,0.00,0.0,US,0,True,True,False,False,True,40,Ranger Rick Jr.,1595.0,4.7,Magazine Subscriptions


In [402]:
# remove NAN
df = df.dropna()
df.head()

,id,price,discount,country,quantity,missOverview,missDiscount,missCountry,missQuantity,missPrice,rank,name,ratings_count,rating,category
0,B071FXZBMV,8.99,0.00,US,30,False,True,False,False,False,31,"OHill Cable Clips,16 Pack Black Adhesive Cord ...",45032.0,4.5,Electronics
1,B096WLN8XJ,44.00,0.00,US,2,True,True,False,False,False,38,2021 Topps Series 2 Baseball Blaster Box,203.0,4.7,Sports Collectibles
2,B09B2QR6LR,17.90,0.00,US,7,True,True,False,False,False,81,2021 Panini Prizm Draft Picks Football CELLO p...,21.0,4.1,Sports Collectibles
4,B003I7HS5M,0.00,0.00,US,0,True,True,False,False,True,40,Ranger Rick Jr.,1595.0,4.7,Magazine Subscriptions
5,B005HNZ5T6,11.60,13.36,US,90,False,False,False,False,False,69,"uni-ball 207 Retractable Gel Pens, Bold Point ...",28045.0,4.7,Office Products


In [403]:
# define top seller: rank < 50
df['label'] = 0
top_idx = df['rank'] < 50
df[top_idx] = df[top_idx].assign(label=1)
df.head()

,id,price,discount,country,quantity,missOverview,missDiscount,missCountry,missQuantity,missPrice,rank,name,ratings_count,rating,category,label
0,B071FXZBMV,8.99,0.00,US,30,False,True,False,False,False,31,"OHill Cable Clips,16 Pack Black Adhesive Cord ...",45032.0,4.5,Electronics,1
1,B096WLN8XJ,44.00,0.00,US,2,True,True,False,False,False,38,2021 Topps Series 2 Baseball Blaster Box,203.0,4.7,Sports Collectibles,1
2,B09B2QR6LR,17.90,0.00,US,7,True,True,False,False,False,81,2021 Panini Prizm Draft Picks Football CELLO p...,21.0,4.1,Sports Collectibles,0
4,B003I7HS5M,0.00,0.00,US,0,True,True,False,False,True,40,Ranger Rick Jr.,1595.0,4.7,Magazine Subscriptions,1
5,B005HNZ5T6,11.60,13.36,US,90,False,False,False,False,False,69,"uni-ball 207 Retractable Gel Pens, Bold Point ...",28045.0,4.7,Office Products,0


# ML Model

In [458]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.utils import shuffle


In [459]:
df = df.sample(frac=1, random_state=42)

In [500]:

X = df[['price', 'discount' , 'rating', 'ratings_count']].to_numpy()
y = df['label'].to_numpy()

In [501]:
N_train = 400

train_acc = []
test_acc = []

for i in range(50):
    X, y = shuffle(X, y, random_state=0)
    train_X, test_X = X[:N_train], X[N_train:]
    train_y, test_y = y[:N_train], y[N_train:]
    clf = LogisticRegression(random_state=0).fit(train_X, train_y)
    train_acc.append(clf.score(train_X, train_y))
    test_acc.append(clf.score(test_X, test_y))

print(f'train acc mean:{np.mean(train_acc)}')
print(f'test acc mean:{np.mean(test_acc)}')

train acc mean:0.5658
test acc mean:0.5502139037433156
